In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-65c6fbe3-9da8-df49-9f7e-3f3bb55ae785)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/Graduation_Project/CheXpert-v1.0-small.zip > /dev/null

In [4]:
!pip install timm > /dev/null

In [5]:
import cv2
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

import torch 
from torch import nn
from torchsummary import summary

import timm
from torch.optim import Adam, lr_scheduler

from myGen import MyGen

from sklearn.metrics import roc_auc_score

In [6]:
train = pd.read_csv('/content/CheXpert-v1.0-small/train.csv')

In [7]:
train.loc[:, train.columns[5:]] = train.loc[:, train.columns[5:]].fillna(0)

zeros = ['Cardiomegaly', 'Consolidation', 'Pleural Effusion']
train.loc[:, zeros] = train.loc[:, zeros].replace({-1:0})
ones = ['Edema', 'Atelectasis']
train.loc[:, ones] = train.loc[:, ones].replace({-1:1})

train = train[['Path', 'Frontal/Lateral', 'Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']].copy() 

In [8]:
train.to_csv('/content/CheXpert-v1.0-small/train_edited.csv', index=False)

In [9]:
save_weights_path = '/content/drive/MyDrive/Graduation_Project/eff_normal.pth' #edit
load_weights_path = '/content/drive/MyDrive/Graduation_Project/eff_normal.pth' #edit

In [10]:
model = timm.create_model('efficientnet_b0', num_classes=5, pretrained=True) #edit
model = model.cuda()

summary(model, (3, 224, 224), batch_size=64, device='cuda')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [64, 32, 112, 112]             864
       BatchNorm2d-2         [64, 32, 112, 112]              64
              SiLU-3         [64, 32, 112, 112]               0
            Conv2d-4         [64, 32, 112, 112]             288
       BatchNorm2d-5         [64, 32, 112, 112]              64
              SiLU-6         [64, 32, 112, 112]               0
            Conv2d-7              [64, 8, 1, 1]             264
              SiLU-8              [64, 8, 1, 1]               0
            Conv2d-9             [64, 32, 1, 1]             288
          Sigmoid-10             [64, 32, 1, 1]               0
    SqueezeExcite-11         [64, 32, 112, 112]               0
           Conv2d-12         [64, 16, 112, 112]             512
      BatchNorm2d-13         [64, 16, 112, 112]              32
         Identity-14         [64, 16, 1

In [11]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [12]:
root = '/content/'

trainGen = MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/train_edited.csv', 
                 image_size=224, #edit
                 frontal=True)

valGen =  MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/valid.csv', 
                 image_size=224,  #edit
                 frontal=True)

trainloader =  torch.utils.data.DataLoader(trainGen, batch_size=64, shuffle=True)
testloader =  torch.utils.data.DataLoader(valGen, batch_size=64, shuffle=False)

# define loss & optimizer
Loss = nn.BCELoss()

optimizer = Adam(model.parameters(), lr=1e-3)

scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Decay LR by a factor of 0.5 every 5 epochs

#model.load_state_dict(torch.load(load_weights_path))

# training
const_threshold = 5

val_auc_mean = 0
best_val_auc = 0
const_cnt = 0

for epoch in range(5):

    for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      y_pred = torch.sigmoid(y_pred)
      loss = Loss(y_pred, train_labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
      if idx % 400 == 0:
         model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for data in testloader:
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  y_pred = torch.sigmoid(y_pred)
                  test_pred.append(y_pred.cpu().detach().numpy())            
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 
              model.train()

              if best_val_auc < val_auc_mean:
                  const_cnt = 0
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
                  best_val_auc = val_auc_mean
                  torch.save(model.state_dict(), save_weights_path)
              else:
                  const_cnt += 1
                  scheduler.step()            

              if (const_cnt > const_threshold):
                  const_cnt = 0 
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
                  model.load_state_dict(torch.load(save_weights_path))
                               
              lr = get_lr(optimizer)

              print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f, lr=%f' %(epoch, idx, val_auc_mean, best_val_auc, lr))

Epoch=0, BatchID=0, Val_AUC=0.5389, Best_Val_AUC=0.5389, lr=0.001000
Epoch=0, BatchID=400, Val_AUC=0.8435, Best_Val_AUC=0.8435, lr=0.001000
Epoch=0, BatchID=800, Val_AUC=0.8744, Best_Val_AUC=0.8744, lr=0.001000
Epoch=0, BatchID=1200, Val_AUC=0.8760, Best_Val_AUC=0.8760, lr=0.001000
Epoch=0, BatchID=1600, Val_AUC=0.8755, Best_Val_AUC=0.8760, lr=0.001000
Epoch=0, BatchID=2000, Val_AUC=0.8793, Best_Val_AUC=0.8793, lr=0.001000
Epoch=0, BatchID=2400, Val_AUC=0.8900, Best_Val_AUC=0.8900, lr=0.001000
Epoch=0, BatchID=2800, Val_AUC=0.8700, Best_Val_AUC=0.8900, lr=0.001000
Epoch=1, BatchID=0, Val_AUC=0.8679, Best_Val_AUC=0.8900, lr=0.001000
Epoch=1, BatchID=400, Val_AUC=0.8552, Best_Val_AUC=0.8900, lr=0.001000
Epoch=1, BatchID=800, Val_AUC=0.8665, Best_Val_AUC=0.8900, lr=0.001000
Epoch=1, BatchID=1200, Val_AUC=0.8819, Best_Val_AUC=0.8900, lr=0.000500
Epoch=1, BatchID=1600, Val_AUC=0.8887, Best_Val_AUC=0.8900, lr=0.000500
Epoch=1, BatchID=2000, Val_AUC=0.8870, Best_Val_AUC=0.8900, lr=0.000500
Ep

In [13]:
root = '/content/'
testGen =  MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/valid.csv', 
                 image_size=224, 
                 frontal=True)

testloader =  torch.utils.data.DataLoader(testGen, batch_size=1, shuffle=False)

In [14]:
model.load_state_dict(torch.load(save_weights_path))

<All keys matched successfully>

In [15]:
model.eval()
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for data in testloader:
        test_data, test_labels = data
        test_data = test_data.cuda()
        y_pred = model(test_data)
        y_pred = torch.sigmoid(y_pred)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_labels.numpy())
  
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)

In [16]:
labels = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']

results = pd.DataFrame(index=labels)


scores = []
for i in range(5):
    score = roc_auc_score(test_true[:, i], test_pred[:, i])
    scores.append(score)
    
results['AUC'] = scores

In [17]:
results

,AUC
Cardiomegaly,0.845811
Edema,0.937202
Consolidation,0.921875
Atelectasis,0.818583
Pleural Effusion,0.928442


In [18]:
results['AUC'].mean()

0.8903826277593293